In [10]:
import requests
import json
import prettytable
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2 
import io
from numpy import asarray
from numpy import savetxt

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
file_metro = "data_files/bls_metro_2020_clean.csv"
bls_metro_2020_jobtitle_df = pd.read_csv(file_metro)
bls_metro_2020_jobtitle_df.head()

,Unnamed: 0,area,area_title,prim_state,occ_code,occ_title,o_group,tot_emp,emp_prse,jobs_1000,loc_quotient,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90
0,0,10180,"Abilene, TX",TX,00-0000,All Occupations,total,66060.0,1.9,1000.000,1.00,20.64,42930.0,1.8,9.13,11.40,16.41,24.82,36.23,18990.0,23710.0,34130.0,51620.0,75370.0
1,1,10180,"Abilene, TX",TX,11-0000,Management Occupations,major,2910.0,4.5,44.068,0.77,42.87,89160.0,2.2,17.38,26.74,36.37,51.68,73.88,36150.0,55620.0,75640.0,107500.0,153670.0
2,2,10180,"Abilene, TX",TX,11-1021,General and Operations Managers,detailed,1320.0,7.4,20.036,1.19,40.38,83990.0,3.5,12.67,24.02,32.96,50.11,72.93,26350.0,49960.0,68550.0,104230.0,151700.0
3,4,10180,"Abilene, TX",TX,11-2030,Public Relations and Fundraising Managers,detailed,40.0,31.5,0.549,0.94,45.93,95540.0,20.7,28.71,32.78,36.24,41.23,83.81,59710.0,68180.0,75380.0,85760.0,174320.0
4,5,10180,"Abilene, TX",TX,11-3010,Administrative Services and Facilities Managers,detailed,140.0,12.3,2.045,0.92,37.79,78600.0,3.5,24.93,28.28,35.06,45.45,55.25,51850.0,58820.0,72920.0,94530.0,114920.0


In [4]:
bls_metro_2020_jobtitle_df.drop(['Unnamed: 0'] , axis=1, inplace=True)

bls_metro_2020_jobtitle_df.head(5)

,area,area_title,prim_state,occ_code,occ_title,o_group,tot_emp,emp_prse,jobs_1000,loc_quotient,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90
0,10180,"Abilene, TX",TX,00-0000,All Occupations,total,66060.0,1.9,1000.000,1.00,20.64,42930.0,1.8,9.13,11.40,16.41,24.82,36.23,18990.0,23710.0,34130.0,51620.0,75370.0
1,10180,"Abilene, TX",TX,11-0000,Management Occupations,major,2910.0,4.5,44.068,0.77,42.87,89160.0,2.2,17.38,26.74,36.37,51.68,73.88,36150.0,55620.0,75640.0,107500.0,153670.0
2,10180,"Abilene, TX",TX,11-1021,General and Operations Managers,detailed,1320.0,7.4,20.036,1.19,40.38,83990.0,3.5,12.67,24.02,32.96,50.11,72.93,26350.0,49960.0,68550.0,104230.0,151700.0
3,10180,"Abilene, TX",TX,11-2030,Public Relations and Fundraising Managers,detailed,40.0,31.5,0.549,0.94,45.93,95540.0,20.7,28.71,32.78,36.24,41.23,83.81,59710.0,68180.0,75380.0,85760.0,174320.0
4,10180,"Abilene, TX",TX,11-3010,Administrative Services and Facilities Managers,detailed,140.0,12.3,2.045,0.92,37.79,78600.0,3.5,24.93,28.28,35.06,45.45,55.25,51850.0,58820.0,72920.0,94530.0,114920.0


In [7]:
#seperate job titles for 2020 all state 
Jobs_titles_2020 =bls_metro_2020_jobtitle_df['occ_title'].unique()
Jobs_titles_2020

array(['All Occupations', 'Management Occupations',
       'General and Operations Managers',
       'Public Relations and Fundraising Managers',
       'Administrative Services and Facilities Managers',
       'Computer and Information Systems Managers',
       'Industrial Production Managers',
       'Transportation, Storage, and Distribution Managers',
       'Human Resources Managers', 'Construction Managers',
       'Food Service Managers', 'Medical and Health Services Managers',
       'Social and Community Service Managers',
       'Personal Service Managers, All Other; Entertainment and Recreation Managers, Except Gambling; and Managers, All Other',
       'Business and Financial Operations Occupations',
       'Buyers and Purchasing Agents', 'Compliance Officers',
       'Cost Estimators', 'Human Resources Specialists', 'Logisticians',
       'Management Analysts', 'Fundraisers',
       'Training and Development Specialists',
       'Market Research Analysts and Marketing Spec

In [8]:
len(Jobs_list_2020)

734

In [13]:
#create CSV files to be used to load database as needed
#Jobs_titles_2020.to_csv('data_files/Jobs_titles_2020.csv')
#savetxt('Jobs_titles_2020.csv', Jobs_list_2020, delimiter=',')
pd.DataFrame(Jobs_list_2020).to_csv("data_files/Jobs_titles_2020.csv")

In [14]:
# cities list for 2020
area_titles_2020 =bls_metro_2020_jobtitle_df['area_title'].unique()
area_titles_2020

array(['Abilene, TX', 'Aguadilla-Isabela, PR', 'Akron, OH', 'Albany, GA',
       'Albany, OR', 'Albany-Schenectady-Troy, NY', 'Albuquerque, NM',
       'Alexandria, LA', 'Allentown-Bethlehem-Easton, PA-NJ',
       'Altoona, PA', 'Amarillo, TX', 'Ames, IA', 'Anchorage, AK',
       'Ann Arbor, MI', 'Anniston-Oxford-Jacksonville, AL',
       'Appleton, WI', 'Arecibo, PR', 'Asheville, NC',
       'Athens-Clarke County, GA', 'Atlanta-Sandy Springs-Roswell, GA',
       'Atlantic City-Hammonton, NJ', 'Auburn-Opelika, AL',
       'Augusta-Richmond County, GA-SC', 'Austin-Round Rock, TX',
       'Bakersfield, CA', 'Baltimore-Columbia-Towson, MD',
       'Baton Rouge, LA', 'Battle Creek, MI', 'Bay City, MI',
       'Beaumont-Port Arthur, TX', 'Beckley, WV', 'Bellingham, WA',
       'Bend-Redmond, OR', 'Billings, MT', 'Binghamton, NY',
       'Birmingham-Hoover, AL', 'Bismarck, ND',
       'Blacksburg-Christiansburg-Radford, VA', 'Bloomington, IL',
       'Bloomington, IN', 'Bloomsburg-Berwick, P

In [15]:
len(area_titles_2020)

396

In [16]:
#convert numpy array to csv 
pd.DataFrame(area_titles_2020).to_csv("data_files/area_titles_2020.csv")